# Параллельная разработка с библиотекой `asyncio`

## Основы

Здесь сначала введем новый термин - конкурентность (от англ. concurrency). Почти везде слово concurrency на русский переводится как параллелизм, однако это не совсем корректный перевод. Параллелизм - это когда одновременно работают несколько потоков операций. Допустим, у нас есть ПК с 4 ядрами ЦПУ. Тогда параллельно могут работать не более 4 потоков. При этом, если посмотреть на диспетчер задач, то там скорее всего будет числиться гораздо большее число одновременно выполняемых процессов. То есть, операционная система выделяет каждому из потоков этих процессов какой-то квант процессорного времени, чередуя потоки. Это и есть пример конкурентности. 

В отличие от `concurrent.futures` библиотека `asyncio` реализует многозадачность через конкурентность с помощью корутин и использования цикла обработки событий. Начнем наше знакомство с `asyncio` с изучения различий с явной работой с потоками. Для этого реализуем один и тот же функционал двумя способами: сначала через параллелизм, а потом - через конкурентность.

In [1]:
%%writefile spinner_thread.py

import threading
import time
import sys
import itertools

class Signal:
    go = True
    
def spin(msg, signal):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = f'{char} {msg}'
        write(status)
        flush()
        write('\x08' * len(status))
        time.sleep(.1)
        if not signal.go:
            break
    write(' ' * len(status) + '\x08' * len(status))
    
def slow_function():
    time.sleep(3)
    return 42
    
def supervisor():
    signal = Signal()
    spinner = threading.Thread(target=spin, args=('thinking!', signal))
    print(f'spinner object: {spinner}')
    spinner.start()
    result = slow_function()
    signal.go = False
    spinner.join()
    return result
    
def main():
    result = supervisor()
    print(f'Answer: {result}')

if __name__ == '__main__':
    main()

Overwriting spinner_thread.py


А здесь - реализация того же функционала через конкурентность. Исходя из кода становится понятен принцип конкурентности, которым руководствуется `asyncio`. В коде программы функции заменяются корутинами (см. префикс `async` в определении функции), которые вызывают кроме прочего другие корутины (см. префикс `await` перед вызовом функции). Так мы даем `asyncio` понять, в каком месте можно приостановить выполнение функции и перейти к другой функции.  

In [2]:
%%writefile spinner_asyncio.py

import asyncio
import itertools
import sys

async def spin(msg):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|-/\\'):
        status = f'{char} {msg}'
        write(status)
        flush()
        write('\x08' * len(status))
        try:
            await asyncio.sleep(0.1)
        except asyncio.CancelledError:
            break
    write(' ' * len(status) + '\x08' * len(status))

async def slow_function():
    await asyncio.sleep(3)
    return 42

async def supervisor():
    spinner = asyncio.create_task(spin('thinking!'))
    print(f'spinner object: {spinner}')
    result = await slow_function()
    spinner.cancel()
    return result

def main():
    loop = asyncio.get_event_loop()
    result = loop.run_until_complete(supervisor())
    loop.close()
    print(f'Answer = {result}')

if __name__ == '__main__':
    main()

Overwriting spinner_asyncio.py


В коде выше стоит обратить внимание на создание задачи (объект класса `Task`) методом `create_task()` библиотеки `asyncio`. Класс `Task` является дочерним по отношению к классу `asyncio.Future`, который является vis-a-vis класса `concurrent.futures.Future` (см. ноутбук `Futures.ipynb`) и реализует важный функционал. У данных классов есть принципиальное различие: у обоих есть метод `result()`. Если мы вызовем данный метод в незавершившем работу классе `asyncio.Future`, то в отличие от класса `concurrent.futures.Future` он не заблокирует поток в ожидании завершения работы футуры, а создаст исключение `asyncio.InvalidStateError`. Потому что ожидать выполнения футуры нужно через ключевое слово `await`.

А теперь попробуем с помощью `asyncio` повторить загрузчик флагов.

In [4]:
%%writefile flags_asyncio.py

import asyncio

from httpx import AsyncClient

from flags_seq import BASE_URL, save_flag, main

async def download_one(client: AsyncClient, cc: str):
    image = await get_flag(client, cc)
    save_flag(image, f'{cc}.gif')
    print(cc, end=' ', flush=True)
    return cc

async def get_flag(client: AsyncClient, cc: str) -> bytes:
    url = f'{BASE_URL}/{cc}/{cc}.gif'.lower()
    resp = await client.get(url, timeout=5, follow_redirects=True)
    return resp.read()

def download_many(cc_list: list) -> int:
    return asyncio.run(supervisor(cc_list))

async def supervisor(cc_list: list) -> int:
    async with AsyncClient() as client:
        to_do = [download_one(client, cc) for cc in sorted(cc_list)]
        res = await asyncio.gather(*to_do)

    return len(res)

if __name__ == '__main__':
    main(download_many)

Overwriting flags_asyncio.py
